In [1]:
--1. Create a stored procedure to show the number of students per department.[use ITI DB]
create or alter proc Number_of_Student_in_department @dep_num int
as
select isnull(cast(s.Dept_Id as varchar(10)), 'No department') as Department, count(s.st_id) as 'Number of Student'
from Student s
where s.Dept_Id = @dep_num
group by s.Dept_Id

Commands completed successfully.

Total execution time: 00:00:00.014

In [2]:
exec Number_of_Student_in_department 10

(1 row affected)

Total execution time: 00:00:00.040

Department,Number of Student
10,5


In [8]:
/*2.Create a stored procedure that will check for the number of employees in the project 100 if they are more than 3 print a message to the user
“'The number of employees in the project 100 is 3 or more'” if they are less display a message to the user “'The following employees work for
the project p1'” in addition to the first name and last name of each one. [Company DB] */
CREATE OR ALTER PROCEDURE Emp_In_Proj_100 
AS
BEGIN
    DECLARE @emp_count INT, @message VARCHAR(90)

    SELECT @emp_count = COUNT(*)
    FROM Works_for
    WHERE Pno = 100

    IF @emp_count > 3
    BEGIN
        SET @message = 'The number of employees in project 100 is 3 or more'
        SELECT @message AS 'MESSAGE'
    END
    ELSE
    BEGIN
        SET @message = 'The following employees work for project p1:'
        SELECT @message AS 'MESSAGE', e.Fname, e.LName
        FROM Employee e
        JOIN Works_for wf ON e.SSN = wf.ESSn
        WHERE wf.Pno = 100
    END
END

Commands completed successfully.

Total execution time: 00:00:00.005

In [9]:
Emp_In_Proj_100

(1 row affected)

Total execution time: 00:00:00.017

MESSAGE
The number of employees in project 100 is 3 or more


In [5]:
/*Create a trigger to prevent anyone from inserting a new record in the Department table [ITI DB]
Print a message for the user to tell him that he ‘can’t insert a new record in that table’*/
create or alter trigger Prevent_Insert
on department
after insert
as 
begin
select 'You can not insert'
declare @depid int
select @depid = Dept_Id from inserted
delete from Department 
where Dept_Id = @depid

end

Commands completed successfully.

Total execution time: 00:00:00.005

In [6]:
insert into Department (Dept_Id) values(100)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.014

(No column name)
You can not insert


In [7]:
select * from Department d where d.Dept_Id = 100

(0 rows affected)

Total execution time: 00:00:00.005

Dept_Id,Dept_Name,Dept_Desc,Dept_Location,Dept_Manager,Manager_hiredate


In [8]:
use ITI_Company_SD; 

Commands completed successfully.

Total execution time: 00:00:00.001

In [15]:
drop TABLE history

Commands completed successfully.

Total execution time: 00:00:00.002

In [16]:
/*4. This table will be used to audit the update trials on the Hours column (works_for table, Company DB)
Example:
If a user updated the Hours column then the project number, the user name that made that update, 
the date of the modification and the value of the old and the new Hours will be inserted into the Audit table
Note: This process will take place only if the user updated the Hours column
*/

CREATE TABLE history
(
Proj_Number INT , 
User_Name varchar(50) , 
Update_Date DATE , 
Old_Hours int,
New_Hours int
)

Commands completed successfully.

Total execution time: 00:00:00.002

In [17]:
CREATE OR ALTER TRIGGER Updated_Hours
ON [dbo].[Works_for]
AFTER UPDATE
AS
BEGIN
    IF UPDATE(hours)
    BEGIN
        DECLARE @oldhour INT, @newhour INT, @ProNum int
        SELECT @oldhour = Hours FROM deleted
        SELECT @newhour = Hours FROM inserted
        select @ProNum = pno from inserted
        INSERT INTO HISTORY VALUES(@ProNum, SUSER_NAME(), GETDATE(), @oldhour, @newhour)
        /*SELECT wf.Pno, SUSER_NAME(), GETDATE(), @oldhour as 'Old Hours', @newhour as 'New Hours'
        FROM Works_for wf*/
    END
END

Commands completed successfully.

Total execution time: 00:00:00.003

In [18]:
select * from HISTORY

(0 rows affected)

Total execution time: 00:00:00.003

Proj_Number,User_Name,Update_Date,Old_Hours,New_Hours


In [19]:
update Works_for set Hours = 60 where ESSn = 223344

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.007

In [20]:
select * from Works_for where ESSn = 223344

(4 rows affected)

Total execution time: 00:00:00.004

ESSn,Pno,Hours
223344,100,60
223344,200,60
223344,300,60
223344,500,60


In [21]:
select * from HISTORY

(1 row affected)

Total execution time: 00:00:00.004

Proj_Number,User_Name,Update_Date,Old_Hours,New_Hours
100,DESKTOP-4H0R5VG\sohila,2023-12-12,60,60


In [23]:
/*3.Create a stored procedure that will be used in case there is an old employee has left the project and a new one become instead of him.
The procedure should take 3 parameters (old Emp. number, new Emp. number and the project number) and it will be used to update works_for table.[Company DB]*/
select *
from Works_for wf
where ESSn = 11

(1 row affected)

Total execution time: 00:00:00.017

ESSn,Pno,Hours
11,200,NULL


In [32]:
CREATE or ALTER PROCEDURE UpdateWorksFor
    @oldEmpNumber INT,
    @newEmpNumber INT,
    @projectNumber INT
AS
BEGIN
    UPDATE works_for
    SET ESSn = @newEmpNumber
    WHERE essn = @oldEmpNumber
        AND Pno = @projectNumber;
END

Commands completed successfully.

Total execution time: 00:00:00.003

In [33]:
EXEC UpdateWorksFor @oldEmpNumber = 2, @newEmpNumber = 3, @projectNumber = 700;

(0 rows affected)

Total execution time: 00:00:00.010

In [35]:
select *
from Works_for wf
where ESSn = 2 and wf.Pno = 700


(0 rows affected)

Total execution time: 00:00:00.003

ESSn,Pno,Hours


# **Bons**

## **Last Lab tasks**

In [ ]:
--1. Create a scalar function that takes a date and returns the Month name of that date. test (‘1/12/2009’)
CREATE PROCEDURE GetMonthName
    @inputDate DATE
AS
BEGIN
    SELECT DATENAME(MONTH, @inputDate) AS MonthName;
END

In [10]:
GetMonthName '1-10-2020'

(1 row affected)

Total execution time: 00:00:00.006

MonthName
January


In [11]:
--2. Create a multi-statements table-valued function that takes 2 integers and returns the values between them.
--just between a and b
CREATE OR ALTER PROCEDURE InsertNumbers 
    @start INT,
    @end INT   
AS
BEGIN
	DECLARE @result TABLE (Numbers INT)
    DECLARE @current INT = @start + 1

    WHILE @current < @end
    BEGIN
        INSERT INTO @result (Numbers) VALUES (@current)
        SET @current += 1
    END
    select * from @result
END
GO

Commands completed successfully.

Total execution time: 00:00:00.006

In [12]:
EXEC InsertNumbers @start = 1, @end = 10

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(8 rows affected)

Total execution time: 00:00:00.009

Numbers
2
3
4
5
6
7
8
9


In [15]:
------3. Create a tabled valued function that takes Student No and returns Department Name with Student full name.
CREATE OR ALTER PROCEDURE dbo.StudentName_Department 
    @id INT
AS
BEGIN
    SELECT std.St_Id, std.St_Fname + ' ' + std.St_Lname AS 'Full Name', dep.Dept_Name
    FROM Student std
    JOIN Department dep ON dep.Dept_Id = std.Dept_Id
    WHERE std.St_Id = @id
END

Commands completed successfully.

Total execution time: 00:00:00.002

In [16]:
StudentName_Department 1

(1 row affected)

Total execution time: 00:00:00.692

St_Id,Full Name,Dept_Name
1,Ahmed Hassan,SD


In [17]:
/*4.	Create a scalar function that takes Student ID and returns a message to the user (use Case statement)
a.	If the first name and Last name are null then display 'First name & last name are null'
b.	If the First name is null then display 'first name is null'
c.	If the Last name is null then display 'last name is null'
d.	Else display 'First name & last name are not null'*/

create or alter PROC MessageProc
@id int
as
begin
declare @mess varchar(30)
	select @mess = case
		when std.St_Fname is null then 'first name is null'
		when std.St_Lname is null then 'last name is null'
		when (std.St_Lname is null and std.St_Fname is null) then 'First name & last name are null'
		else 'There is no null'
		end 
	from Student std
	where std.St_Id = @id	
	select @mess as 'Message'
	select s.St_Fname, s.St_Lname
	from Student s
	where s.St_Id = @id

end

Commands completed successfully.

Total execution time: 00:00:01.348

In [18]:
MessageProc 1

(1 row affected)

(1 row affected)

Total execution time: 00:00:02.675

Message
There is no null


St_Fname,St_Lname
Ahmed,Hassan


In [19]:
/*5.Create a function that takes an integer that represents the format of the Manager hiring date and displays department
name, Manager Name, and hiring date with this format.  */

create or alter proc ManagerProc
@hiringDate int
as
begin
select dep.Dnum, dep.Dname, dep.MGRSSN, emp.Fname, CONVERT(varchar, dep.[MGRStart Date], @hiringDate) as HiringDate
from Departments dep
join Employee emp
on emp.SSN = dep.MGRSSN
end 


Commands completed successfully.

Total execution time: 00:00:00.003

In [20]:
ManagerProc 101

(3 rows affected)

Total execution time: 00:00:00.005

Dnum,Dname,MGRSSN,Fname,HiringDate
10,DP1,223344,Kamel,10/10/2022
20,DP2,968574,Noha,01/03/2006
30,DP3,512463,Edward,01/06/2006


In [21]:
ManagerProc 102

(3 rows affected)

Total execution time: 00:00:00.016

Dnum,Dname,MGRSSN,Fname,HiringDate
10,DP1,223344,Kamel,2022.10.10
20,DP2,968574,Noha,2006.01.03
30,DP3,512463,Edward,2006.01.06


In [22]:
/*6.	Create multi-statements table-valued function that takes a string
If string='first name' returns student first name
If string='last name' returns student last name 
If string='full name' returns Full Name from student table 
Note: Use the “ISNULL” function
*/


create or alter proc NameProc
@string varchar(20)
as
begin
declare @result table (Name varchar(30))
	declare @name varchar(30)
	
	if @string = 'first name' 
	begin
		insert into @result 
		select ISNULL(St_Fname, 'No first Name Extists')
		from student
	end	
	else if @string = 'last name' 
	begin
		insert into @result
		select ISNULL(St_Lname, 'No last Name Extists' ) 
		from Student
	end
	else if	@string = 'full name' 
	begin
		insert into @result
		select ISNULL(St_Fname, '') + ' ' +  ISNULL(St_Lname, ' ') as FullName
		from Student
	end
	select * from @result
	end


Commands completed successfully.

Total execution time: 00:00:00.006

In [23]:
NameProc 'first name'

(17 rows affected)

(17 rows affected)

Total execution time: 00:00:00.010

Name
Ahmed
Amr
Mona
Ahmed
Khalid
Heba
Ali
Mohamed
Saly
Fady


In [24]:
/*
7.	Write a query that returns the Student No and Student first name without the last char
*/

create or alter proc Std_SubSting
as

select std.St_Id, substring(St_Fname,1, len(St_Fname) - 1)
from Student std

(17 rows affected)

Total execution time: 00:00:00.010

St_Id,(No column name)
1,Ahme
2,Am
3,Mon
4,Ahme
5,Khali
6,Heb
7,Al
8,Mohame
9,Sal
10,Fad


In [25]:
Std_SubSting

(17 rows affected)

Total execution time: 00:00:00.707

St_Id,(No column name)
1,Ahme
2,Am
3,Mon
4,Ahme
5,Khali
6,Heb
7,Al
8,Mohame
9,Sal
10,Fad


In [27]:

/*1. Create a function that takes project number and display all employees in this project*/

create or alter PROC projectNumberProc
@projNum int

as 
begin
select emp.*
from Employee emp
join Works_for wf
on emp.SSN = wf.ESSn
where wf.Pno = @projNum
end

Commands completed successfully.

Total execution time: 00:00:00.004

In [28]:
projectNumberProc 200

(2 rows affected)

Total execution time: 00:00:00.013

Fname,Lname,SSN,Bdate,Address,Gender,Salary,Superssn,Dno,NewSalary
NULL,NULL,11,NULL,NULL,NULL,NULL,NULL,NULL,NULL
Kamel,Mohamed,223344,1970-10-15 00:00:00.000,38 Mohy el dien abo el Ezz St.Cairo,M,1800,321654,10,1900


In [29]:
projectNumberProc 100

(5 rows affected)

Total execution time: 00:00:00.004

Fname,Lname,SSN,Bdate,Address,Gender,Salary,Superssn,Dno,NewSalary
NULL,NULL,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL
NULL,NULL,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL
NULL,NULL,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL
Ahmed,Ali,112233,1995-01-01 00:00:00.000,15 Ali fahmy St.Giza,M,1400,223344,NULL,1500
Kamel,Mohamed,223344,1970-10-15 00:00:00.000,38 Mohy el dien abo el Ezz St.Cairo,M,1800,321654,10,1900


In [30]:
------------Bons-----------
--1.write a Query that computes the increment in salary that arises if the salary of employees increased by any value.
CREATE OR ALTER PROCEDURE UpdateSalary
@id int
AS
BEGIN
    UPDATE Instructor
    SET Salary += 200
    OUTPUT (INSERTED.Salary - DELETED.Salary) * 100 / INSERTED.Salary AS 'Percent of Salary'
    WHERE Ins_Id = @id
END



Commands completed successfully.

Total execution time: 00:00:00.004

In [31]:
UpdateSalary 1

(1 row affected)

Total execution time: 00:00:00.016

Percent of Salary
5.8823
